# Primeiros Passos para programar um jogo
* Instalar o pygame no computador: pip install pygame.
* Precisamos importar 3 bibliotecas: pygame, os e random.
* Os serve para pegar o caminho das imagens.
* Random para gerar numeros aleatorios pois os canos não podem aparecer na mesma posição toda hora.

In [1]:
import pygame # cria a interface do jogo
import os # controla o computador por linha de comando (os é operacional sistem)
import random # sugere valores aleatórios (random em inglês é aleatório)

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Definindo algumas constantes
* Por padrão, uma definição de constante no pyhton é escrito com letras maiúsculas.
* Constantes não devem ser mudadas nunca, oras, são constantes.
* toda imagem que se roda no pygame é escrita como: pygame.image.load("nome_do_arquivo.png")
* contudo precisamos colocar o sistema operacional para entrar na pasta iamgens (imgs) e ler os arquivos. Exemplo: os.path.join('imgs', 'pipe')
* para não ficar muito pequeno, vamos aumentar as imagens em 2 vezes com comando: pygame.transform.scale2x()
* Juntando tudo fica: pygame.transform.scale2x(pygame.image.load(os.path.join('imgs', 'pipe.png')))
* Precisamos de uma constante para desenhar a pontuação. Exemplo: pygame.font.SysFont('times', 12)

In [2]:
# largura da tela e altura será 500 x 800.
TELA_LARGURA = 500
TELA_ALTURA = 800

# lista de imagens do jogo.
IMAGEM_CANO = pygame.transform.scale2x(pygame.image.load(os.path.join('imgs', 'pipe.png')))
IMAGEM_CHAO = pygame.transform.scale2x(pygame.image.load(os.path.join('imgs', 'base.png')))
IAMGEM_BACKGROUND = pygame.transform.scale2x(pygame.image.load(os.path.join('imgs', 'bg.png')))

# Lista de imagens do personagem
IMAGENS_PASSAROS = [
    pygame.transform.scale2x(pygame.image.load(os.path.join('imgs', 'bird1.png'))),
    pygame.transform.scale2x(pygame.image.load(os.path.join('imgs', 'bird2.png'))),
    pygame.transform.scale2x(pygame.image.load(os.path.join('imgs', 'bird3.png'))),    
]                           

# Inicilizando a fonte
pygame.font.init()
FONTE_PONTOS = pygame.font.SysFont('arial', 50)


# Criando os objetos do jogo.
* Devemos nos perguntar o que existe no jogo? ou o que se movimenta no jogo?
* R: o pássaro, o cano e o chão
* Cada um desses objeto será uma classe do python. Ou seja, teremos 3 classes
* Cada classe deve ter atributos e métodos que os objetos podem ter
* atributos é o que tem o objeto (Adjetivo)
* métodos é o que o objeto pode fazer (verbo)
* Exemplo: O cano tem comprimento, e distancia até o chão como atributos. Pode caminhar para esquerda a uma velocidade X (métodos)
* vamos utilizar a formula do sorvete da físisca para dar movimento vertical ao personagem. Olha que beleza!!!!!
* ![Descrição da imagem](imgs/sorvete.png)
* MUITO DO QUE ESTÁ NO CÓDIGO A SEGUIR FOI POR TENTATIVA E ERRO, PORTANTO UM JOGO SÓ FICA PERFEITO COM VÁRIAS VERSÕES!!!!!!



In [3]:
# Criação da classe (objeto) do python
class Passaro:
    IMGS = IMAGENS_PASSAROS
    # animação de rotação
    ROTACAO_MAXIMA = 25
    VELOCIDADE_ROTACAO = 20
    TEMPO_ANIMACAO = 5
    
# === Método de inciar uma classe. ================
    def __init__(self, x, y):
        self.x = x
        self.y = y  
        self.angulo = 0
        self.velocidade = 0
        self.altura = self.y
        self.tempo = 0
        self.contagem_imagem = 0
        self.imagem = self.IMGS[0]
    
    def pular(self):
        self.velocidade = -7.5
        self.tempo = 0 #será inserido na formula do sorvete. para ter animação de queda
        self.altura = self.y
    
# === Método de animar e dar física ao passaro. ===
    def mover(self):
        # calcular o deslocamento
        self.tempo += 1 # serve para atualizarr enquanto temos um novo frame
        deslocamento = 1.2 * (self.tempo**2) + self.velocidade * self.tempo
                
        # restringir o deslocamento
        if deslocamento > 16:
            deslocamento = 16 # limita movimento absurdos do passaro, sem isso o passaro acelera infinitamente.
        elif deslocamento < 0:
            deslocamento -= 2 # ganho extra quando o passaro pula, pode fazer sem. facilitar a jogabilidade do jogo
        
        #precisamo movimentar o passaro:
        self.y += deslocamento
        
        # o angulo de rotação do pássaro: quando passaro pula vira para cima enquanto tá pulando, so vira para baixo no ponto maximo
        if deslocamento < 0 or self.y < (self.altura + 50): # self.altura + 50 é a diferença de onde tava e o pulo que deu.
            # se o passaro tá pulando rotação maxima
            if self.angulo < self.ROTACAO_MAXIMA:
                self.angulo = self.ROTACAO_MAXIMA
        # se o passaro não tá pulando eu deixo ele rotacionar a vontade até chegar a 90º em relação a horizontal
        # o passaro não deve ficar rotacioando para tras
        else: 
            if self.angulo > -90:
                self.angulo -= self.VELOCIDADE_ROTACAO
                # perceba que estamos diminuindo com a notação -= self.VELOCIDADE_ROTACAO
                
                
# === Método de desenhar o passaro. ===============
    def desenhar(self, tela):
        
        # definir qual imagem do passaro vai usar.
        self.contagem_imagem += 1                
        if self.contagem_imagem < self.TEMPO_ANIMACAO:
            self.imagem = self.IMGS[0] # asa pra cima
        elif self.contagem_imagem < self.TEMPO_ANIMACAO*2:
            self.imagem = self.IMGS[1] # asa pro meio
        elif self.contagem_imagem < self.TEMPO_ANIMACAO*3:
            self.imagem = self.IMGS[2] # asa pra baixo           
        elif self.contagem_imagem < self.TEMPO_ANIMACAO*4:
            self.imagem = self.IMGS[1] # asa pro meio
        elif self.contagem_imagem < self.TEMPO_ANIMACAO*4 + 1:
            self.imagem = self.IMGS[0] # asa pra cima
            self.contagem_imagem = 0 # RETORNO O CONTADOR DE IMAGENS PARA ZERO FRAME.
        
        # se o passaro estive caindo, a imgem com asa parada e não bate asas.
        if self.angulo <= -80:
            self.image = self.IMGS[1]
            self.contagem_imagem = self.TEMPO_ANIMACAO*2 # Ajuste fino: Quando tá caindo a proxima batida de asa será para baixo e não para cima
            
        
        # desenhar a imagem
        # para desenhar uma imagem na tela do jogo, você deve construir um retângulo virtual do tamanho da imagem, e jogar a imagem do personagem lá dentro. (Colisão)
        # ademias precisa saber onde etão as posições centrais do retangulo criado para dizer para o pygame onde será desenhado seu personagem.
        # O centro da imamgem e o centro do retangulo virtual sempre precisa está casados.    
        # Qualquer coisa que vc desenha no pygame, o ponto de referencia é sempre topleft (topo esquerdo). Mosstrar em desenho.
        
        imagem_rotacionada = pygame.transform.rotate(self.imagem, self.angulo) # a imagem que queremos desenhar não pode ser estática sempre numa direção.
        pos_centro_imagem = self.imagem.get_rect(topleft = (self.x, self.y)).center # serve para pegar a posição central da imagem e jogar no retangulo virtual, que cola na tela. Explicar que topleft é (0,0) vc quer o centro
        retangulo = imagem_rotacionada.get_rect(center = pos_centro_imagem) # comando para desenhar um retangulo em volta da sua imamge, esse retangulo vamos colar na tela, precisa da posição central, linha anterior.
        tela.blit(imagem_rotacionada, retangulo.topleft) # Desenha na tela que será passada futuramente.
        
    
    # função que criar uma mascara semelhante a uma derivada de pixel para estabelecer a colisão (cabe o desenho da "Derivada")
    def get_mask(self):
        return pygame.mask.from_surface(self.imagem)

In [4]:
class Cano:
    
    DISTANCIA = 200 # Distancia entre canos: 200 pixel    
    VELOCIDADE = 5 # velocidade com que os canos se movimentam.   
    
    def __init__(self, x):
        self.x = x #onde o cano vai começar
        self.altura = 0 # posição em y onde o cano irá está
        self.pos_topo = 0 
        self.pos_base = 0
        self.CANO_BASE = IMAGEM_CANO
        self.CANO_TOPO = pygame.transform.flip(IMAGEM_CANO, False, True) #False para espelhar em X e True para espelhar Y
        self.passou = False #identificar se  o passaro passou ou não do cano (contabilizar). Quando cria o parâmetro é falso.
        self.definir_altura() # toda vez que rodar a classe do cano iniciar o método do cano.
        
    # Vai gerar o valor da altura do cano.
    def definir_altura(self):
        self.altura = random.randrange(50, 450) # vai randomizar de 0 à altura de tela, (cabe explicar pq do 50 a 450, para não criar cano na base e no topo)
        self.pos_topo = self.altura - self.CANO_TOPO.get_height()
        self.pos_base = self.altura + self.DISTANCIA

    def mover(self):
        self.x -= self.VELOCIDADE
        
    def desenhar(self, tela):
        tela.blit(self.CANO_BASE, (self.x, self.pos_base)) #Desenhando o cano do topo        
        tela.blit(self.CANO_TOPO, (self.x, self.pos_topo)) #Desenhando o cano do topo        
        # === Fazer em aula =======
        #desenhando o cano da base
        
        
    # === Método de colisões. =====
    def colidir(self, passaro):
        #criando as mascaras.
        passaro_mask = passaro.get_mask()
        topo_mask = pygame.mask.from_surface(self.CANO_TOPO)
        base_mask = pygame.mask.from_surface(self.CANO_BASE)

        # vamos calcular as distâncias entre as posições do passaro em relação aos canos da base e do topo
        distancia_topo =  (self.x - passaro.x, self.pos_topo - round(passaro.y))
        distancia_base =  (self.x - passaro.x, self.pos_base - round(passaro.y))
        
        # verificando a colisão (vai ser True ou False o resultado)
        topo_ponto = passaro_mask.overlap(topo_mask, distancia_topo)
        base_ponto = passaro_mask.overlap(base_mask, distancia_base)
        
        if base_ponto or topo_ponto:
            return True
        else:
            False 

In [5]:
# é necessário explicar a logica dos 2 chãos
# será apresentado 1 encavalodo ao outro

class Chao:
    VELOCIDADE = 5
    LARGURA = IMAGEM_CHAO.get_width() #tamanho do chão
    IMAGEM = IMAGEM_CHAO

    def __init__(self, y):
        self.y = y
        self.x1 = 0 # x do primeiro chão
        self.x2 = self.x1 + self.LARGURA # x do segundo chão

    def mover(self):
        # primeiro movo os 2 chãos
        self.x1 -= self.VELOCIDADE # chão 0 está adquirindo velocidade
        self.x2 -= self.VELOCIDADE # chão 1 está adquirindo velocidade
        
        # se o alguns dos chão saiu da tela, voltar com ele para o início
        if self.x1 + self.LARGURA < 0:
            self.x1 = self.x1 + self.LARGURA
        
        # se o alguns dos chão saiu da tela, voltar com ele para o início
        if self.x2 + self.LARGURA < 0:
            self.x2 = self.x2 + self.LARGURA
        
    def desenhar(self, tela):
            tela.blit(self.IMAGEM, (self.x1, self.y)) #Desenhando o chão 1       
            tela.blit(self.IMAGEM, (self.x2, self.y)) #Desenhando o chão 2       

In [6]:
# função que criar a tela e pega todos os objetos criados.
# passaros no plural por conta da IA que vamos utilizar.

def desenhar_tela(tela, passaros, chao, canos, pontuacao):
    tela.blit(IAMGEM_BACKGROUND, (0,0)) # desenahndo o fundo da tela na posição 0, 0 da tela.

    for passaro in passaros:
        passaro.desenhar(tela) # já existe um método desenhar em pássaros. 
    
    for cano in canos:
        cano.desenhar(tela) # desenhamos sempre 2 canos sucessivos na tela

    texto = FONTE_PONTOS.render(f'Pontuação: {pontuacao}', 1, (255, 255, 255)) # cria o texto de pontuação, 1 = sem pixealização, o texto é branco
    tela.blit(texto, (TELA_LARGURA - 10 - texto.get_width(), 10)) # desenhando o texto no fim da tela - 10 pixel - a largura do texto em X, em y 10 pixel abaixo.

    chao.desenhar(tela)

    pygame.display.update()  

In [ ]:
# criando a classe principal para de fato rodar o jogo

def main():
    passaros = [Passaro(230, 350)]
    chao = Chao(730)
    canos = [Cano(700)]
    tela = pygame.display.set_mode((TELA_LARGURA, TELA_ALTURA)) # criando de fato a tela
    pontos = 0
    relogio = pygame.time.Clock() # Criando o rológio interno do jogo

    rodando = True
    inicio = False
    while rodando:
        relogio.tick(30) # precisa dizer quantos frames/s precisa andar, nesse caso é 30.

        # criando métodos de interação com o jogo
        for evento in pygame.event.get():
            
            # evento de sair do jogo
            if evento == pygame.QUIT:
                rodando = False
                pygame.quit()
                quit()
            
            # evento de pular (barra de espaço). Se o tipo de evento for apertar uma tecla para baixo e se a tecla for barra de espaço
            if evento.type == pygame.KEYDOWN:
                if evento.key == pygame.K_SPACE:
                    for passaro in passaros: # aqui estamos lidando com uma lista de passaros que será mudado depois para adicionar a inteligencia artificial com muitos passaros.
                        passaro.pular()
                        inicio = True

        
        if inicio == True:
            # mover as coisas
            for passaro in passaros:
                passaro.mover()        
            chao.mover()

            # Adição de canos
            # primeiro avalia se o passaro colidiu no cano, depois se ele passou pelo cano, e por ultimo se o cano já saiu da dela
            adicionar_cano = False
            remover_cano = []

            for cano in canos:
                for i, passaro in enumerate(passaros):
                    if cano.colidir(passaro):
                        passaro.pop(i)
                    if not cano.passou and passaro.x > cano.x:
                        cano.passou = True
                        adicionar_cano = True
                cano.mover()
                #move o cano e coloca ele numa lista de canos removíveis se já passou da tela
                if cano.x + cano.CANO_TOPO.get_width() < 0:
                    remover_cano.append(cano)
            
            if adicionar_cano:
                pontos = +1
                canos.append(Cano(600))
            for cano in remover_cano:
                cano.pop(cano)

            for i, passaro in enumerate(passaros):
                if (passaro.y + passaro.imagem.get_height()) > chao.y or passaro.y < 0:
                    passaros.pop(i)

        desenhar_tela(tela, passaros, chao, canos, pontos) # ultima coisa que temos que fazer


main()

AttributeError: 'Cano' object has no attribute 'pop'

: 